In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
import pandas as pd
import numpy as np
from pydataset import data

from pydataset import data
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import min, max
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit, round

### 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be `language`

- View the schema of the dataframe

- Output the shape of the dataframe

- Show the first 5 records in the dataframe

In [2]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "language": [
                "Python",
                "R",
                "JavaScript",
                "PHP",
            ]
        }
    )
)

textdf.show(truncate=False)

+----------+
|language  |
+----------+
|Python    |
|R         |
|JavaScript|
|PHP       |
+----------+



In [3]:
textdf.printSchema()

root
 |-- language: string (nullable = true)



In [4]:
print((textdf.count(), len(textdf.columns)))


(4, 1)


In [5]:
textdf.show(5)

+----------+
|  language|
+----------+
|    Python|
|         R|
|JavaScript|
|       PHP|
+----------+



### 3. Load the mpg dataset as a spark dataframe.

    a. Create 1 column of output that contains a message like the one below:

> `The 1999 audi a4 has a 4 cylinder engine.`

    For each vehicle.

    b. Transform the trans column so that it only contains either manual or auto.

In [6]:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [17]:
mpg.select(concat(lit('The '), mpg.year, lit(' '), mpg.manufacturer, \
    lit(' '), mpg.model, lit(' has a '), mpg.cyl, \
    lit(' cylinder engine.'),).alias('vehicle_info')).show(truncate = False)

+--------------------------------------------------------------+
|vehicle_info                                                  |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

In [26]:
mpg.select(
    regexp_extract('trans', r'^(\w+), 1'.alias('trans'),
    when(mpg.trans.like('auto%'), 'auto').otherwise('manual'),
).show()

SyntaxError: unexpected EOF while parsing (<ipython-input-26-85eff680c5c3>, line 4)

## 3. Load the tips dataset as a spark dataframe.

    a. What percentage of observations are smokers?

    b. Create a column that contains the tip percentage

    c. Calculate the average tip percentage for each combination of sex and smoker.

## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to farenheight.


- Which month has the most rain, on average?


- Which year was the windiest?


- What is the most frequent type of weather in January?


- What is the average high and low tempurature on sunny days in July in 2013 and 2014?


- What percentage of days were rainy in q3 of 2015?


- For each year, find what percentage of days it rained (had non-zero precipitation).

In [27]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [29]:
weather.select(
    col('temp_max') * 9 / 5 + 32,
    col('temp_min') * 9 / 5 + 32,
).show()

+---------------------------+---------------------------+
|(((temp_max * 9) / 5) + 32)|(((temp_min * 9) / 5) + 32)|
+---------------------------+---------------------------+
|                      55.04|                       41.0|
|                      51.08|                      37.04|
|                      53.06|                      44.96|
|                      53.96|                      42.08|
|                      48.02|                      37.04|
|                      39.92|                      35.96|
|                      44.96|                      37.04|
|                       50.0|                      37.04|
|                      48.92|                       41.0|
|         42.980000000000004|                      33.08|
|         42.980000000000004|                      30.02|
|         42.980000000000004|                      28.94|
|                       41.0|                      26.96|
|                      39.92|                      33.08|
|             

In [31]:
weather = (weather
    .withColumn('temp_max', col('temp_max') * 9 / 5 + 32)
    .withColumn('temp_min', col('temp_min') * 9 / 5 + 32))

In [34]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05|          1.3|   48.02|   37.04| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [ ]:
(weather
    .withColumn('month')
    )